# Min-Code: How little code will suffice to write min-cut from scratch?

## 2024-08-26: Intro

<a href="https://adventofcode.com/2023/day/25">Day 25</a> of <a href="https://adventofcode.com/2023">Advent of Code 2023</a> describes a highly-interconnected network that can be split in two by cutting three specific wires. Our main job is to identify those three wires. This is known as finding a <a href="https://en.wikipedia.org/wiki/Minimum_cut">minimum cut</a>.

When a simple algorithm is needed, I often like to write it from scratch each time, using only built-ins and standard libraries. (Inspirations include the <a href="https://www.reddit.com/r/adventofcode/">Advent of Code subreddit</a>, <a href="https://github.com/norvig/pytudes#pytudes-index-of-jupyter-ipython-notebooks">Peter Norvig</a>, and <a href="https://www.youtube.com/watch?v=j6VSAsKAj98">David Beazley</a>.) By the sixth or seventh time I wrote a breadth-first search, I did it without any conscious doubt about how I was doing it. But that was BFS.

I read about maxflows and mincuts in Sedgewick's <a href="https://algs4.cs.princeton.edu/">Algorithms</a>. Their Java code for maxflow problems includes several classes but is quite general. Some classes contain many attributes and methods that I wouldn't need to solve Day 25. I wondered whether I could simplify Sedgewick's approach enough to rewrite it from scratch, in Python, and still get a decent night's sleep. My main problem was that I didn't really understand the <a href="https://en.wikipedia.org/wiki/Ford%E2%80%93Fulkerson_algorithm">Ford–Fulkerson algorithm</a>, and how to identify the residual network in Sedgewick's implementation so as to find a minimum cut. As of starting this project, I still don't.

Instead, to collect the last of my 50 stars that night — disclaimer: it was January 30, not December 24 — I took a brute-force approach and eventually came up with a simplifying assumption that, although not guaranteed to produce the correct answer, did so in under ten seconds runtime. A few months later, I used <a href="https://networkx.org/documentation/stable/">NetworkX</a> to write an almost trivial solution to the problem, letting the `minimum_edge_cut` and `connected_components` functions do the hard work. NetworkX seems pretty great, and there's probably no reason to avoid using it if I'm already working in Python.

But now it's time to return to my original objective. Can I simplify min-cut enough so that I can write it from scratch as needed?

## 2024-08-27: Initial solution

Before diving any deeper, I'll describe a cleaned-up version of my <a href="https://github.com/RobinFiveWords/AdventOfCode/blob/main/2023/day25.py">initial solution</a>. If we accept that our starting graph (network) has just one connected component — from any given vertex we can reach every other vertex — then we can test any potential solution by removing its three edges (wires) from the graph and rechecking whether from any given vertex we can reach every other vertex. To check the modified graph, we can arbitrarily pick a vertex and do a depth-first search, counting how many vertices we reach. If it is less than the total number of vertices, we have our solution.

I couldn't just brute-force it. My puzzle input had 1,514 vertices connected by 3,385 edges, meaning I would potentially have to test
$$
\frac{3{,}385 \times 3{,}384 \times 3{,}383}{3 \times 2 \times 1} = 38{,}751{,}723{,}720 \nonumber
$$
combinations of three edges. So I looked for a way to reduce the number of possible solutions I would consider.

The assumption I came up with was that the three edges in the solution would connect vertices — let's call these the solution vertices — that were "closer" to all other vertices, on average, than most other vertices were. In other words, if for any given vertex I added up the shortest number of edges that would need to be traversed to reach each other vertex, the solution vertices would be among the vertices with the lowest sums.

For my reasoning, let's consider one of the edges in the solution to be $X{\textendash}Y$, connecting vertices $X$ and $Y$. In the modified graph, let's say $X$ is in connected component $A$, and $Y$ is in connected component $B$. Now let's consider all the vertices in $A$ other than $X$ for which $X{\textendash}Y$ is the closest solution edge. For these vertices in $A$, their shortest path to reach any vertex in $B$ is through $X$, which means they are farther from every vertex in $B$ than $X$ is. If $X$ is closer to many other vertices than many others are, perhaps on average it is closer to _all_ other vertices than _most_ others are.

For each vertex, I did a breadth-first search to compute the sum of its distances from each other vertex. I then identified any edge in the graph whose vertices were both among the 100 with the lowest total distance. There were 105 such edges, meaning I would only have to test
$$
\frac{105 \times 104 \times 103}{3 \times 2 \times 1} = 187{,}460 \nonumber
$$
combinations of three edges. One of these turned out to be the solution.